In [ ]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Button, Div
from bokeh.layouts import layout
output_notebook()


In [ ]:
LIGHT_GREY = (240<<24) | (240<<16) | (240<<8) | 255
SKIP_ITERS = 50
MAX_ITERS = 200
MAX_ALPHA = 255

def bifurcation(width, height, x_min, x_max, y_min, y_max):
    img = np.full((height, width), LIGHT_GREY, dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((height, width, 4))
    x_delta = (x_max - x_min) / width
    y_delta = (y_max - y_min) / height
    for col in range(width):
        c = x_min + col * x_delta
        x = 0
        for i in range(MAX_ITERS):
            x = x*x + c
            if x == float('inf') or x == float('-inf'):
                break
            if i >= SKIP_ITERS:
                row = round((x - y_min) / y_delta)
                if 0 <= row < height:
                    view[row, col, 0] = 0
                    view[row, col, 1] = 0
                    view[row, col, 2] = 255  # blue
    return img


In [ ]:
def modify_doc(doc):
    div = Div(text='''Bifurcation Diagram of x^2 + c and the Orbits of x=0''')
    refresh_but = Button(label="Refresh", button_type="success")
    
    p = figure(x_range=(-2.5, 2.5), y_range=(-2.5,2.5), 
               tooltips=[("x", "$x"), ("y", "$y")])
    source = ColumnDataSource(data=dict(image=[], x=[], y=[], dw=[], dh=[]))
    p.image_rgba(image='image', x='x', y='y', dw='dw', dh='dh', source=source)
    
    def refresh():
        x_min = p.x_range.start
        x_max = p.x_range.end
        y_min = p.y_range.start
        y_max = p.y_range.end
        source.data = dict(
            image=[bifurcation(
                p.inner_width, p.inner_height,
                x_min, x_max,
                y_min, y_max)],
            x=[x_min],
            y=[y_min],
            dw=[x_max - x_min],
            dh=[y_max - y_min]
        )
    refresh_but.on_click(refresh)
    
    l = layout([div], [p], [refresh_but])
    doc.add_root(l)

show(modify_doc)
